# Post Processing Notebook 2: Graphs for E+ Double Room Experiments 

* 02/13/23

In [3]:
from ladybug.sql import SQLiteResult
from ladybug.analysisperiod import AnalysisPeriod as ap
import numpy as np
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import itertools

In [100]:
dir = "../rosseRoomModel/test13/run/eplusout.sql"

In [101]:
sql_path = os.path.join(dir)
sqld = SQLiteResult(sql_path)

In [102]:
sqld.data_collections_by_output_name("Electricity:Facility")[-1].total

482.4566741762167

In [103]:
sqld.available_outputs

('Zone Air Temperature',
 'Site Direct Solar Radiation Rate per Area',
 'Electricity:Facility',
 'Surface Inside Face Temperature',
 'Site Outdoor Air Drybulb Temperature',
 'People Occupant Count',
 'Electricity:Facility',
 'People Convective Heating Energy',
 'People Air Temperature',
 'People Total Heating Rate',
 'People Air Relative Humidity',
 'Site Outdoor Air Wetbulb Temperature',
 'Surface Outside Face Temperature',
 'People Total Heating Energy',
 'Site Diffuse Solar Radiation Rate per Area',
 'Site Outdoor Air Drybulb Temperature',
 'Zone Air Relative Humidity')

## Zone vs Outdoor Air Analysis

In [104]:
# July 20 noon (actually like 7am, but ok) to July 23 noon 
exp01_ap = ap(st_month=7, st_day=20, end_month=7, end_day=23,  is_leap_year=True)
# exp01_ap.datetimes

In [105]:
sqld.data_collections_by_output_name("Zone Air Temperature")

[Hourly Continuous Data Collection
 7/1 to 9/30 between 0 and 23 @1*
 Temperature (C)
 ...2208 values...,
 Hourly Continuous Data Collection
 7/1 to 9/30 between 0 and 23 @1*
 Temperature (C)
 ...2208 values...]

In [106]:
sqld.data_collections_by_output_name("Zone Air Temperature")[0].header

Temperature (C)
7/1 to 9/30 between 0 and 23 @1*
type: Zone Air Temperature
System: THERMAL ZONE A

In [107]:

# rooms["zone_b"].append(sqld.data_collections_by_output_name("Zone Air Temperature")[1])

In [108]:
zone_air = sqld.data_collections_by_output_name("Zone Air Temperature")[0]
zone_air_exp01 = zone_air.filter_by_analysis_period(exp01_ap) #.datetimes

In [109]:
amb_air = sqld.data_collections_by_output_name("Site Outdoor Air Drybulb Temperature")[0]
amb_air_exp01 = amb_air.filter_by_analysis_period(exp01_ap)

In [110]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=zone_air_exp01.datetimes,
    y=zone_air_exp01.values, 
    mode='lines+markers',
    name="Zone Air Temperature"
))
fig.add_trace(go.Scatter(
    x=amb_air_exp01.datetimes,
    y=amb_air_exp01.values, 
    mode='lines+markers',
    name="Site Outdoor Air Temperature"
))

fig.update_layout(title='rr_SurfaceConst021223',
                   xaxis_title='Dates',
                   yaxis_title='Temperature (ºC)')


## Surface Analysis

In [111]:
surf_names = []
for surf in out_surf:
    surf_names.append(surf.header.to_dict()["metadata"]["Surface"])
surf_names

['BACK A',
 'FRONT A',
 'SIDE01 A',
 'SIDE02 A',
 'SURFACE 2',
 'FLOOR A',
 'CEILING A',
 'SUB SURFACE 1',
 'BACK B',
 'FRONT B',
 'SIDE01 B',
 'SIDE02 B',
 'SURFACE 10',
 'FLOOR B',
 'CEILING B',
 'SUB SURFACE 2']

In [112]:
def make_data_dict(rooms, room, ix):
    """
    rooms: dictionary with correct formatting 
    room: room letter, A or B
    ix: number, 0 or 1, corresponding to order of room in SQL output  
    """
    rooms[f"zone_{room}"]["air"].append(sqld.data_collections_by_output_name("Zone Air Temperature")[ix].filter_by_analysis_period(exp01_ap))
    rooms[f"zone_{room}"]["amb"].append(sqld.data_collections_by_output_name("Site Outdoor Air Drybulb Temperature")[0].filter_by_analysis_period(exp01_ap))

    in_surf = sqld.data_collections_by_output_name("Surface Inside Face Temperature")
    out_surf = sqld.data_collections_by_output_name("Surface Outside Face Temperature")

    for outs, ins in zip(out_surf, in_surf):
        if f" {room}" in outs.header.to_dict()["metadata"]["Surface"]:
            rooms[f"zone_{room}"]["out"].append(outs.filter_by_analysis_period(exp01_ap))
        if f" {room}" in ins.header.to_dict()["metadata"]["Surface"]:
            rooms[f"zone_{room}"]["in"].append(ins.filter_by_analysis_period(exp01_ap))

    return rooms

#     order = [1, 2, 3, 0, 4, 5]
# rooms["zone_A"]["in"] = [x for _, x in sorted(zip(order, rooms["zone_A"]["in"]))]

In [113]:
rooms = {
    "zone_A" : {
        "air": [],
        "amb": [],
        "out": [],
        "in": []
    },
    "zone_B" : {
        "air": [],
        "amb": [],
        "out": [],
        "in": []
    }
}

rooms = make_data_dict(rooms, "A", 0)
rooms = make_data_dict(rooms, "B", 1)
rooms["zone_B"]["air"]

[Hourly Continuous Data Collection
 7/20 to 7/23 between 0 and 23 @1*
 Temperature (C)
 ...96 values...]

In [114]:
surf_names_a = [a.header.to_dict()["metadata"]["Surface"] for a in rooms["zone_A"]["in"]]
surf_names_b = [a.header.to_dict()["metadata"]["Surface"] for a in rooms["zone_B"]["in"]]
alt_names = ["Back", "Front", "Side01", "Side02", "Floor", "Top"]

surf_names = ["A"] + alt_names + ["B"] + alt_names

In [115]:
fig = make_subplots(rows=2, cols=7, shared_yaxes=True, shared_xaxes=True, subplot_titles=surf_names 
)


## zone air temp, in vs out 
for ix, room in enumerate(rooms.values()):
    showlegendbool = True if ix == 0 else False
    fig.add_trace(go.Scatter(
        x=room["air"][0].datetimes,
        y=room["air"][0].values, 
        mode='lines',
        name="Zone Air Temperature",
        showlegend=showlegendbool,
        marker_color="darksalmon"
    ), row=ix+1, col=1)
    fig.add_trace(go.Scatter(
        x=room["amb"][0].datetimes,
        y=room["amb"][0].values, 
        mode='lines',
        name="Site Outdoor Air Temperature",
        showlegend=showlegendbool,
        marker_color="dimgrey"
    ), row=ix+1, col=1)

for ix, room in enumerate(rooms.values()):
    
    for i in range(len(room["in"])):
        showlegendbool = True if i == 0 and ix == 0 else False
        fig.add_trace(go.Scatter(
            x=room["in"][i].datetimes,
            y=room["in"][i].values, 
            mode='lines',
            name="Indoor Surface",
            showlegend=showlegendbool,
            marker_color="darkblue"
        ), row=ix+1, col=i+2)
        fig.add_trace(go.Scatter(
            x=room["out"][i].datetimes,
            y=room["out"][i].values, 
            mode='lines',
            name="Outdoor Surface",
            showlegend=showlegendbool,
            marker_color="pink"
        ), row=ix+1, col=i+2)

fig.show()

In [116]:
["Back", "Front", "Side01", "Side02", "Floor", "Top"]

['Back', 'Front', 'Side01', 'Side02', 'Floor', 'Top']